In [153]:
class record:
    
    @property
    def id(self):
        return self.id_
        
    @id.setter
    def id(self, value):
        self.id_ = value
    
    @property
    def country(self):
        return self.country_
        
    @country.setter
    def country(self, value):
        self.country_ = value
        
    @property
    def action(self):
        return self.action_
        
    @action.setter
    def action(self, value):
        self.action_ = value
        
    @property
    def center(self):
        return self.center_
        
    @center.setter
    def center(self, value):
        self.center_ = value
        
    @property
    def processingType(self):
        return self.processingType_
        
    @processingType.setter
    def processingType(self, value):
        self.processingType_ = value
        
    @property
    def fillingDate(self):
        return self.fillingDate_
        
    @fillingDate.setter
    def fillingDate(self, value):
        self.fillingDate_ = value
        
    @property
    def receivedDate(self):
        return self.receivedDate_
        
    @receivedDate.setter
    def receivedDate(self, value):
        self.receivedDate_ = value
        
    @property
    def receiptNumber(self):
        return self.receiptNumber_
        
    @receiptNumber.setter
    def receiptNumber(self, value):
        self.receiptNumber_ = value
        
    @property
    def rfeRecieved(self):
        return self.rfeRecieved_
        
    @rfeRecieved.setter
    def rfeRecieved(self, value):
        self.rfeRecieved_ = value
        
    @property
    def rfeRecievedDate(self):
        return self.rfeRecievedDate_
        
    @rfeRecievedDate.setter
    def rfeRecievedDate(self, value):
        self.rfeRecievedDate_ = value
        
    @property
    def reasonForRfe(self):
        return self.reasonForRfe_
        
    @reasonForRfe.setter
    def reasonForRfe(self, value):
        self.reasonForRfe_ = value
        
    @property
    def rfeRepliedDate(self):
        return self.rfeRepliedDate_
        
    @rfeRepliedDate.setter
    def rfeRepliedDate(self, value):
        self.rfeRepliedDate_ = value
        
    @property
    def applicationStatus(self):
        return self.applicationStatus_
        
    @applicationStatus.setter
    def applicationStatus(self, value):
        self.applicationStatus_ = value
        
    @property
    def approvalDate(self):
        return self.approvalDate_
        
    @approvalDate.setter
    def approvalDate(self, value):
        self.approvalDate_ = value
        
    @property
    def tnStatus(self):
        return self.tnStatus_
        
    @tnStatus.setter
    def tnStatus(self, value):
        self.tnStatus_ = value
        
    @property
    def mostRecentLud(self):
        return self.mostRecentLud_
        
    @mostRecentLud.setter
    def mostRecentLud(self, value):
        self.mostRecentLud_ = value
        
    @property
    def note(self):
        return self.note_
        
    @note.setter
    def note(self, value):
        self.note_ = value
        
    @property
    def state(self):
        return self.state_
        
    @state.setter
    def state(self, value):
        self.state_ = value
        

In [154]:
url = 'http://www.trackitt.com/usa-immigration-trackers/h1b/page/'

In [155]:
# setup 
import re
import urllib2
from bs4 import BeautifulSoup
from datetime import datetime

proxy = urllib2.ProxyHandler({'http': 'totproxy01.nomura.com'})
opener = urllib2.build_opener(proxy)
urllib2.install_opener(opener)

testurl = url + '1'
response = urllib2.urlopen(testurl)
src = response.read()
soup = BeautifulSoup(src, 'html.parser')

#find the number of pages

#print(soup.prettify())

clear = soup.find_all('div', { 'class' : 'paginator' })[0]

#print(clear.prettify())

select = clear.find_all('select')[0]

#print(select.prettify())

count = [int(r.get_text()) for r in select.find_all('option')]

In [165]:
debug = True


recs = []
    
for i in count:
    
    print('loading page {0}'.format(i))
    pageUrl = url + str(i)
    response = urllib2.urlopen(testurl)
    src = response.read()
    soup = BeautifulSoup(src, 'html.parser')
    #print(soup.prettify())
    
    # first get the header to retrieve field position
    head = soup.find_all('thead')[0]
    #print(head.prettify())
    
    headers = head.findAll("th", { 'valign' : 'top' })
    #print(headers[0].prettify())
    
    _map = [r.findAll('nobr')[0].get_text() for r in headers]
    
    if debug:
        print(_map)
    
    # once we get the right mapping, decifer the body
    body = soup.find_all('tbody')[1]
    #print(body.prettify())
    
    rows = body.findAll("tr", { 'class' : 'regulartrackerrow' })
    #print(rows[0])

    rId = re.compile('row_([0-9]+)', re.IGNORECASE)

    for r in rows:
        
        print('parse new record')
        if debug:
            #print(r)
            pass
            
        rec = record()

        # get the id from the header
        rec.id = int(rId.search(r['id']).group(1))
        recs.append(rec)

        # from https://stackoverflow.com/questions/35465182/how-to-find-all-divs-whos-class-starts-with-a-string-in-beautifulsoup
        fields = r.select('td[class^=trackercell]')
        #fields = r.findAll('td', { 'class' : lambda value: value and value.startswith('trackercell') })

        try:
            rec.country = fields[_map.index(u'Nationality ')].get_text()
            rec.action = fields[_map.index(u'\nApplication Type\n\n')].get_text()
            rec.center = fields[_map.index(u'\nService Center\n\n')].get_text()
            rec.processingType = fields[_map.index(u'\nProcessing Type\n\n')].get_text()
            
            pos = _map.index(u'\nApplication Filed\n\n')
            if len(fields[pos].get_text()) > 0:
                rec.fillingDate = datetime.strptime(fields[pos].get_text(), '%m/%d/%Y').date()
            else:
                rec.fillingDate = None

            pos = _map.index(u'\nUSCIS Received Date\n\n')
            if len(fields[pos].get_text()) > 0:
                rec.receivedDate = datetime.strptime(fields[pos].get_text(), '%m/%d/%Y').date()
            else:
                rec.receivedDate = None

            pos = _map.index(u'\nReceipt Number\n\n')
            if len(fields[pos].get_text()) > 0:
                rec.receiptNumber = fields[pos].get_text()
            else:
                rec.receiptNumber = None

            pos = _map.index(u'\nReceipt Number\n\n')
            if fields[9].get_text() == 'no':
                rec.rfeRecieved = False
            elif fields[9].get_text() == 'yes':
                rec.rfeRecieved = True
            elif fields[9].get_text() == '':
                rec.rfeRecieved = None
            else:
                raise ValueError('rfe recieved field corrupted')

            if len(fields[10].get_text()) > 0:
                rec.rfeRecievedDate = datetime.strptime(fields[10].get_text(), '%m/%d/%Y').date()
            else:
                rec.rfeRecievedDate = None

            if len(fields[11].get_text()) > 0:
                rec.reasonForRfe = fields[11].get_text()
            else:
                rec.reasonForRfe = None

            if len(fields[12].get_text()) > 0:
                rec.rfeRepliedDate = datetime.strptime(fields[12].get_text(), '%m/%d/%Y').date()
            else:
                rec.rfeRepliedDate = None

            rec.applicationStatus = fields[13].get_text()

            if len(fields[14].get_text()) > 0:
                rec.approvalDate = datetime.strptime(fields[14].get_text(), '%m/%d/%Y').date()
            else:
                rec.approvalDate = None

            if len(fields[15].get_text()) > 0:
                rec.tnStatus = fields[15].get_text()
            else:
                rec.tnStatus = None

            if len(fields[16].get_text()) > 0:
                rec.mostRecentLud = fields[16].get_text()
            else:
                rec.mostRecentLud = None

            if len(fields[19].get_text()) > 0:
                rec.note = fields[19].get_text()
            else:
                rec.note = None

            if len(fields[20].get_text()) > 0:
                rec.state = fields[20].get_text()
            else:
                rec.state = None
                
            recs.append(rec)
            
        except ValueError as ex:
            print('failed to parse record:\n {0}'.format(r))
            # continue
    
    # only one row
    debug = False


loading page 1
[u'Watch ', u'\nUsername\n', u'Comments ', u'Nationality ', u'\nApplication Type\n\n', u'\nService Center\n\n', u'\nProcessing Type\n\n', u'\nApplication Filed\n\n', u'\nUSCIS Received Date\n\n', u'\nReceipt Number\n\n', u'\nRFE Received?\n\n', u'\nRFE Received Date\n\n', u'\nReason for RFE\n\n', u'\nRFE Replied Date\n\n', u'\nApplication Status\n\n', u'\nApproval/Denial Date\n\n', u'\nTN Status\n\n', u'\nMost Recent LUD\n\n', u'\nTotal Processing Time\n\n', u'\nDays Elapsed\n\n', u'\nNotes\n\n', u'\nState ', u'\nCase Added to Tracker ', u'\nLast Updated ']
parse new record
parse new record
parse new record
parse new record
parse new record
failed to parse record:
 <tr class="regulartrackerrow" id="row_21029" onclick='selectRow(21029, "regulartrackerrow");'>
<td align="center" class="fixedrowregular highlight_21029" onclick='selectRow("21029","regulartrackerrow");'><input alt="Watch this case (requires login)" onclick="window.location.href='http://www.trackitt.com/login'

KeyboardInterrupt: 